In [5]:
using Random, Plots
using Zygote, ForwardDiff
using OrdinaryDiffEq, DiffEqSensitivity
using LinearAlgebra
using Statistics
using ProgressBars, Printf
using Flux
using Flux.Optimise: update!
using Flux.Losses: mae, mse
using BSON: @save, @load
using DelimitedFiles
using YAML

In [6]:
pwd()

"C:\\Users\\Suraj\\Documents\\Biomass_run"

In [7]:
ENV["GKSwstype"] = "100"

"100"

In [14]:
conf = YAML.load_file("./config.yaml")

Dict{Any, Any} with 15 entries:
  "lr_max"        => 0.002
  "grad_max"      => 100.0
  "maxiters"      => 50000
  "n_epoch"       => 5000
  "lr_min"        => 1.0e-5
  "is_restart"    => false
  "lr_decay"      => 0.5
  "lr_decay_step" => 500
  "nc"            => 1
  "expr_name"     => "5s8r1c-01"
  "nr"            => 8
  "n_plot"        => 10
  "ns"            => 6
  "lb"            => 1.0e-8
  "w_decay"       => 1.0e-8

In [15]:
expr_name = conf["expr_name"]
fig_path = string("./results_catalyst/", expr_name, "/figs")
ckpt_path = string("./results_catalyst/", expr_name, "/checkpoint")
config_path = "./results_catalyst/$expr_name/config.yaml"

is_restart = Bool(conf["is_restart"])
ns = Int64(conf["ns"])
nr = Int64(conf["nr"])
nc = Int64(conf["nc"])
nss = Int64(conf["ns"]) - Int64(conf["nc"])
lb = Float64(conf["lb"])
n_epoch = Int64(conf["n_epoch"])
n_plot = Int64(conf["n_plot"])
grad_max = Float64(conf["grad_max"])
maxiters = Int64(conf["maxiters"])

lr_max = Float64(conf["lr_max"])
lr_min = Float64(conf["lr_min"])
lr_decay = Float64(conf["lr_decay"])
lr_decay_step = Int64(conf["lr_decay_step"])
w_decay = Float64(conf["w_decay"])

1.0e-8

In [16]:
llb = lb;
global p_cutoff = -1.0

const l_exp = 1:9
n_exp = length(l_exp)

l_train = []
l_val = []
for i = 1:n_exp
    j = l_exp[i]
    if !(j in [1,5,9])
        push!(l_train, i)
    else
        push!(l_val, i)
    end
end

opt = Flux.Optimiser(
 ExpDecay(lr_max, lr_decay, length(l_train) * lr_decay_step, lr_min),
 ADAMW(lr_max, (0.9, 0.999), w_decay),
);

#opt = Flux.Optimise.ADAMW(lr_max,(0.9,0.999), w_decay)

In [17]:
l_train, l_val, n_exp

(Any[2, 3, 4, 6, 7, 8], Any[1, 5, 9], 9)

In [18]:
if !is_restart
    if ispath(fig_path)
        rm(fig_path, recursive = true)
    end
    if ispath(ckpt_path)
        rm(ckpt_path, recursive = true)
    end
end

if ispath("./results_catalyst") == false
    mkdir("./results_catalyst")
end

if ispath("./results_catalyst/$expr_name") == false
    mkdir("./results_catalyst/$expr_name")
end

if ispath(fig_path) == false
    mkdir(fig_path)
    mkdir(string(fig_path, "/conditions"))
end

if ispath(ckpt_path) == false
    mkdir(ckpt_path)
end

cp("./config.yaml", config_path; force=true)

"./results_catalyst/5s8r1c-01/config.yaml"

In [19]:
function load_exp(filename)
    exp_data = readdlm(filename)  # [t, T, m]
    index = indexin(unique(exp_data[:, 1]), exp_data[:, 1])
    exp_data = exp_data[index, :]
    exp_data[:, 3] = exp_data[:, 3] / maximum(exp_data[:, 3])
    return exp_data
end

load_exp (generic function with 1 method)

In [20]:
l_exp_data = [];
l_exp_info = zeros(Float64, length(l_exp), 3);
for (i_exp, value) in enumerate(l_exp)
    filename = string("Exp_data5/expdata_no", string(value), ".txt")

    exp_data = Float64.(load_exp(filename))

    #exp_data = exp_data[1:80, :]
    
    # if value == 22
    #     exp_data = exp_data[1:90, :]
    # elseif value == 23
    #     exp_data = exp_data[1:100, :]
    # elseif value == 24
    #     exp_data = exp_data[1:100, :]
    #end

    push!(l_exp_data, exp_data)
    l_exp_info[i_exp, 1] = exp_data[1, 2] # initial temperature, K
end
l_exp_info[:, 2] = readdlm("Exp_data5/beta.txt")[l_exp];
l_exp_info[:, 3] = (readdlm("Exp_data5/cata_conc.txt")[l_exp]);

In [21]:
l_exp_data[4]

76×3 Matrix{Float64}:
    0.0  461.61  1.0
 1000.0  503.42  0.994208
 1250.0  513.9   0.988417
 1500.0  524.35  0.978764
 1600.0  528.47  0.974903
 1650.0  530.59  0.972973
 1700.0  532.67  0.969112
 1750.0  534.74  0.967181
 1800.0  536.8   0.96332
 1850.0  538.89  0.959459
 1900.0  540.95  0.955598
 1950.0  543.05  0.951737
 2000.0  545.12  0.947876
    ⋮            
 4718.0  658.32  0.214286
 4918.0  666.67  0.212355
 5118.0  675.01  0.210425
 5318.0  683.32  0.208494
 5518.0  691.63  0.206564
 5718.0  699.98  0.202703
 5918.0  708.29  0.200772
 6118.0  716.6   0.198842
 6318.0  724.9   0.194981
 6518.0  733.22  0.19305
 6718.0  741.57  0.19112
 6918.0  749.94  0.189189

In [22]:
l_exp_info

9×3 Matrix{Float64}:
 497.56   2.5  0.0
 523.22  10.0  0.0
 490.64   2.5  0.001
 461.61   2.5  0.01
 517.04  10.0  0.001
 490.7   10.0  0.01
 473.2    5.0  0.0
 473.17   5.0  0.001
 473.25   5.0  0.01

In [23]:
Random.seed!(12345)
np = nr * (ns + nc + 3) + 1
p = randn(Float64, np) .* 5.e-2;
p[1:nr] .+= 0.8;  # w_lnA
#p[nr*(nss+1)+1:nr*(ns+1)].*= 10            #w_cat_in
p[nr*(ns+1)+1:nr*(ns+nc+1)] .= 0         #w_cat_out
p[nr*(ns+nc+1)+1:nr*(ns+nc+2)] .+= 0.8;  # w_Ea
p[nr*(ns+nc+2)+1:nr*(ns+nc+3)] .+= 0.1;  # w_b
p[end] = 0.1;  #slope

In [24]:
function p2vec(p)
    slope = p[end] .* 1.e1
    w_b = p[1:nr] .* (slope * 10.0)
    w_b = clamp.(w_b, 0, 50)

    w_out = reshape(p[nr+1:nr*(nss+1)], nss, nr)
    @. w_out[1, :] = clamp(w_out[1, :], -3.0, 0.0)
    @. w_out[end, :] = clamp(abs(w_out[end, :]), 0.0, 3.0)

    if p_cutoff > 0.0
        w_out[findall(abs.(w_out) .< p_cutoff)] .= 0.0
    end

    w_out[nss-1:nss-1, :] .=
        -sum(w_out[1:nss-2, :], dims = 1) .- sum(w_out[nss:nss, :], dims = 1)

    w_cat_in = p[nr*(nss+1)+1:nr*(ns+1)]
    w_cat_out = p[nr*(ns+1)+1:nr*(ns+nc+1)]*0
    
    #w_cat_out = - sign.(w_cat_in) .* abs.(w_cat_out);
    w_cat_in = abs.(w_cat_in)
    
    
    if p_cutoff > 0.0
        w_cat_in[findall(abs.(w_cat_in) .< p_cutoff)] .= 0.0
    end
    
    w_in_Ea = abs.(p[nr*(ns+nc+1)+1:nr*(ns+nc+2)].* (slope * 100.0))
    w_in_Ea = clamp.(w_in_Ea, 0.0, 300.0)

    w_in_b = abs.(p[nr*(ns+nc+2)+1:nr*(ns+nc+3)])


    w_in = vcat(clamp.(-w_out, 0.0, 3.0), w_cat_in', w_in_Ea', w_in_b')
    w_out = vcat(w_out, w_cat_out')
    
    return w_in, w_b, w_out
end

p2vec (generic function with 1 method)

In [348]:
# expr_name1 = "5s8r1c-02"
# ckpt_path1 = string("./results_catalyst/", expr_name1, "/checkpoint")
# @load string(ckpt_path1, "/mymodel.bson") p

In [25]:
function display_p(p)
    w_in, w_b, w_out = p2vec(p)
    println("\n species (column) reaction (row)")
    println("w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out")
    show(stdout, "text/plain", round.(hcat(w_in', w_b, w_out'), digits = 2))
    # println("\n w_out")
    # show(stdout, "text/plain", round.(w_out', digits=3))
    println("\n")
end
display_p(p)


 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×15 Matrix{Float64}:
  0.01  0.0   0.0   0.15  0.0  0.03  83.95  0.08  7.27  -0.01   0.02   0.03  -0.15  0.11  0.0
  0.01  0.08  0.04  0.0   0.0  0.07  80.93  0.06  7.79  -0.01  -0.08  -0.04   0.13  0.0   0.0
  0.01  0.1   0.05  0.0   0.0  0.02  72.92  0.12  8.42  -0.01  -0.1   -0.05   0.12  0.04  0.0
  0.0   0.01  0.06  0.0   0.0  0.06  68.25  0.11  8.21  -0.0   -0.01  -0.06   0.05  0.02  0.0
  0.05  0.0   0.05  0.0   0.0  0.01  81.05  0.01  7.51  -0.05   0.0   -0.05  -0.0   0.1   0.0
 -0.0   0.0   0.0   0.01  0.0  0.02  75.51  0.14  7.25   0.0   -0.0    0.01  -0.01  0.0   0.0
 -0.0   0.04  0.05  0.0   0.0  0.0   80.93  0.11  8.31   0.0   -0.04  -0.05   0.07  0.02  0.0
  0.09  0.03  0.0   0.0   0.0  0.03  84.52  0.05  7.3   -0.09  -0.03   0.03   0.06  0.02  0.0



In [26]:
function getsampletemp(t, T0, beta)
    
    T = clamp.((T0 + beta / 60 * t), 0, 873.15)   # K/min to K/s

    return T
end

getsampletemp (generic function with 1 method)

In [27]:
const R = -1.0 / 8.314e-3  # universal gas constant, kJ/mol*K
@inbounds function crnn!(du, u, p, t)
    logX = @. log(clamp(u, lb, 10.0))
    T = getsampletemp(t, T0, beta)
    w_in_x = w_in' * vcat(logX, R / T, log(T))
    du .= w_out * (@. exp(w_in_x + w_b))
end

crnn! (generic function with 1 method)

In [28]:
tspan = [0.0, 1.0];
u0 = zeros(ns);
u0[1] = 1.0;
prob = ODEProblem(crnn!, u0, tspan, p, abstol = lb)

ODEProblem with uType Vector{Float64} and tType Float64. In-place: true
timespan: (0.0, 1.0)
u0: 6-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [29]:
condition(u, t, integrator) = u[1] < lb * 10.0
affect!(integrator) = terminate!(integrator)
_cb = DiscreteCallback(condition, affect!)

alg = TRBDF2();
#alg = RK4();
#alg = TRBDF2();
#alg = AutoTsit5(Rosenbrock23(autodiff=false));
#alg = Rosenbrock23(autodiff=false);
#alg = AutoVern7(Rodas5P(autodiff = false));
#sense = BacksolveAdjoint(checkpointing=true; autojacvec=ZygoteVJP());
sense = ForwardSensitivity(autojacvec = true)

ForwardSensitivity{0, true, Val{:central}}(true, false)

In [30]:
function pred_n_ode(p, i_exp, exp_data)
    global T0, beta, cat_conc = l_exp_info[i_exp, :]
    global w_in, w_b, w_out = p2vec(p)

    ts = @view(exp_data[:, 1])
    tspan = [ts[1], ts[end]]
    u0[end] = cat_conc 
    sol = solve(
        prob,
        alg,
        tspan = tspan,
        p = p,
        saveat = ts, 
        sensealg = sense,
        maxiters = maxiters
        # callback = _cb,
    )

    if sol.retcode == :Success
        nothing
    else
        #@sprintf("solver failed beta: %.0f ocen: %.2f", beta, exp(ocen))
        @sprintf("solver failed beta: %.0f", beta)
    end
    if length(sol.t) > length(ts)
        # @show exp_data[:, 1]
        # @show sol.t
        return  sol[:, 1:length(ts)]
    else
        return sol
    end
end

pred_n_ode (generic function with 1 method)

In [31]:
function loss_neuralode(p, i_exp)
    exp_data = l_exp_data[i_exp]
    pred = Array(pred_n_ode(p, i_exp, exp_data))
    masslist = sum(clamp.(@view(pred[1:end-1-nc, :]), 0, Inf), dims = 1)'
    gaslist = clamp.(@views(pred[end-nc, :]), 0,  Inf)

    loss = mae(masslist, @view(exp_data[1:length(masslist), 3])) + mae(gaslist, 1 .- @view(exp_data[1:length(masslist), 3]))
    return loss
end

loss_neuralode (generic function with 1 method)

In [356]:
# function loss_neuralode(p, i_exp)
#     exp_data = l_exp_data[i_exp]
#     pred = Array(pred_n_ode(p, i_exp, exp_data))
#     masslist = sum(clamp.(@view(pred[1:end-1-nc, :]), 0, Inf), dims = 1)'
#     gaslist = clamp.(@views(pred[end-nc, :]), 0,  Inf)

#     loss = mse(masslist, @view(exp_data[1:length(masslist), 3])) + mse(gaslist, 1 .- @view(exp_data[1:length(masslist), 3]))
#     return loss
# end

In [32]:
@time loss = loss_neuralode(p, 1)

 16.619802 seconds (45.09 M allocations: 2.649 GiB, 6.86% gc time, 99.58% compilation time)


1.1469355162922197

In [358]:
function plot_sol(i_exp, sol, exp_data, Tlist, cap, sol0 = nothing)
    T0, beta, cat = l_exp_info[i_exp, :]
    #T0, beta = l_exp_info[i_exp, :]
    ts = sol.t / 60.0
    ind = length(ts)
    plt = plot(
        exp_data[:, 1] / 60.0,
        exp_data[:, 3],
        seriestype = :scatter,
        label = "Exp",
    )

    plot!(
        plt,
        ts,
        sum(clamp.(sol[1:end-1-nc, :], 0, Inf), dims = 1)',
        lw = 3,
        legend = :left,
        label = "CRNN",
    )
    
    if sol0 !== nothing
        plot!(
            plt,
            sol0.t / 60,
            sum(sol0[1:end-1-nc, :], dims = 1)',
            label = "initial model",
        )
    end
    xlabel!(plt, "Time [min]")
    ylabel!(plt, "Mass")
    ylims!(0,1.1)
    title!(plt, cap)
    exp_cond = string(
        @sprintf(
            #"T0 = %.1f K \n beta = %.2f K/min",
            "T0 = %.1f K \n beta = %.2f K/min \n [catalyst] = %.2f",
            T0,
            beta,
            cat*100
        )
    )
    annotate!(plt, exp_data[end, 1] / 60.0 * 0.85, 0.4, exp_cond)

    plt2 = twinx()
    plot!(
        plt2,
        exp_data[1:ind, 1] / 60,
        Tlist,
        lw = 2,
        ls = :dash,
        legend = :topright,
        label = "T",
    )
    ylabel!(plt2, "Temp")

    p2 = plot(ts, sol[1, :], lw = 2, legend = :right, label = "Cellulose")
    for i = 2:ns-nc
        plot!(p2, ts, sol[i, :], lw = 2, label = "S$i")
    end
    xlabel!(p2, "Time [min]")
    ylabel!(p2, "Mass")

    plt = plot(plt, p2, framestyle = :box, layout = @layout [a; b])
    plot!(plt, size = (800, 800))
    return plt
end

cbi = function (p, i_exp)
    exp_data = l_exp_data[i_exp]
    sol = pred_n_ode(p, i_exp, exp_data)
    Tlist = similar(sol.t)
    #T0, beta, ocen = l_exp_info[i_exp, :]
    T0, beta = l_exp_info[i_exp, :]
    for (i, t) in enumerate(sol.t)
        Tlist[i] = getsampletemp(t, T0, beta)
    end
    value = l_exp[i_exp]
    plt = plot_sol(i_exp, sol, exp_data, Tlist, "exp_$value")
    png(plt, string(fig_path, "/conditions/pred_exp_$value"))
    return false
end


function plot_loss(l_loss_train, l_loss_val; yscale = :log10)
    plt_loss = plot(l_loss_train, yscale = yscale, label = "train")
    plot!(plt_loss, l_loss_val, yscale = yscale, label = "val")
    plt_grad = plot(list_grad, yscale = yscale, label = "grad_norm")
    xlabel!(plt_loss, "Epoch")
    ylabel!(plt_loss, "Loss")
    xlabel!(plt_grad, "Epoch")
    ylabel!(plt_grad, "Gradient Norm")
    # ylims!(plt_loss, (-Inf, 1e0))
    # ylims!(plt_grad, (-Inf, 1e3))
    #plt_all = plot([plt_loss, plt_grad]..., legend = :top, framestyle=:box)
    # plot!(
    #     plt_all,
    #     size=(800, 400),
    #     xtickfontsize = 11,
    #     ytickfontsize = 11,
    #     xguidefontsize = 12,
    #     yguidefontsize = 12,
    # )
    png(plt_loss, string(fig_path, "/loss_plot"))
    png(plt_grad, string(fig_path, "/grad_plot"))
end

l_loss_train = []
l_loss_val = []
list_grad = []
iter = 1
cb = function (p, loss_train, loss_val, g_norm)
    global l_loss_train, l_loss_val, list_grad, iter
    push!(l_loss_train, loss_train)
    push!(l_loss_val, loss_val)
    push!(list_grad, g_norm)

    #if iter % n_plot == 0
    if all(x -> loss_val < x, l_loss_val[1:end-1])
        display_p(p)
        list_exp = randperm(n_exp)[1]
        @printf(
            "Min Loss train: %.2e val: %.2e",
            minimum(l_loss_train),
            minimum(l_loss_val)
        )
        println("\n update plot ", l_exp[list_exp], "\n")
        for i_exp in list_exp
            cbi(p, i_exp)
        end
        @save string(ckpt_path, "/mymodel.bson") p opt l_loss_train l_loss_val list_grad iter
    end
    
    plot_loss(l_loss_train, l_loss_val; yscale = :log10)
    iter += 1 
end

if is_restart
    @load string(ckpt_path, "/mymodel.bson") p opt l_loss_train l_loss_val list_grad iter
    iter += 1
end

In [359]:
epochs = ProgressBar(iter:n_epoch);
loss_epoch = zeros(Float64, n_exp);
grad_norm = zeros(Float64, n_exp);
for epoch in epochs
    global p
    for i_exp in randperm(n_exp)
        if i_exp in l_val
            continue
        end
        grad = ForwardDiff.gradient(x -> loss_neuralode(x, i_exp), p)
        grad_norm[i_exp] = norm(grad, 2)
        if grad_norm[i_exp] > grad_max
            grad = grad ./ grad_norm[i_exp] .* grad_max
        end
        update!(opt, p, grad)
    end
    for i_exp = 1:n_exp
        loss_epoch[i_exp] = loss_neuralode(p, i_exp)
    end
    loss_train = mean(loss_epoch[l_train])
    loss_val = mean(loss_epoch[l_val])
    grad_mean = mean(grad_norm[l_train])
    set_description(
        epochs,
        string(
            @sprintf(
                "Loss train: %.2e val: %.2e grad: %.2e lr: %.1e",
                loss_train,
                loss_val,
                grad_mean,
                opt[1].eta
                #opt.eta
            )
        ),
    )
    cb(p, loss_train, loss_val, grad_mean)
end

conf["loss_train"] = minimum(l_loss_train)
conf["loss_val"] = minimum(l_loss_val)
YAML.write_file(config_path, conf)

0.0%┣                                             ┫ 0/2.5k [00:00<00:-7, -0s/it]



 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×15 Matrix{Float64}:
  0.04  0.01  0.06  0.0   0.0  0.08  70.3   0.08  7.1   -0.04  -0.01  -0.06   0.09  0.01  0.0
  0.03  0.0   0.0   0.03  0.0  0.04  70.74  0.11  7.6   -0.03   0.0   -0.0   -0.03  0.06  0.0
  0.02  0.04  0.02  0.0   0.0  0.02  79.11  0.05  7.66  -0.02  -0.04  -0.02   0.05  0.04  0.0
  0.01  0.11  0.01  0.0   0.0  0.04  81.38  0.06  6.9   -0.01  -0.11  -0.01   0.1   0.03  0.0
 -0.0   0.03  0.0   0.04  0.0  0.01  81.15  0.07  7.44   0.0   -0.03   0.05  -0.04  0.02  0.0
  0.01  0.02  0.0   0.0   0.0  0.06  71.12  0.16  7.07  -0.01  -0.02   0.01  -0.0   0.02  0.0
 -0.0   0.04  0.0   0.06  0.0  0.01  72.75  0.12  7.73   0.0   -0.04   0.05  -0.06  0.05  0.0
  0.05  0.02  0.02  0.01  0.0  0.08  65.95  0.08  6.93  -0.05  -0.02  -0.02  -0.01  0.1   0.0

Min Loss train: 7.56e-01 val: 7.27e-01
 update plot 6



Loss train: 7.56e-01 val: 7.27e-01 grad: 8.86e+00 lr: 2.0e-03 0.0%┣┫ 1/2.5k [00:26<Inf:Inf, InfGs/it]



 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×15 Matrix{Float64}:
  0.05  0.0   0.05  0.0   0.0  0.07  65.32  0.08  6.67  -0.05  -0.0   -0.05   0.09  0.01  0.0
  0.04  0.0   0.0   0.01  0.0  0.03  65.64  0.12  7.14  -0.04   0.01  -0.0   -0.01  0.05  0.0
  0.03  0.04  0.02  0.0   0.0  0.02  74.04  0.04  7.14  -0.03  -0.04  -0.02   0.06  0.04  0.0
  0.02  0.12  0.01  0.0   0.0  0.05  76.51  0.05  6.4   -0.02  -0.12  -0.01   0.12  0.03  0.0
 -0.0   0.02  0.0   0.05  0.0  0.01  76.1   0.07  6.92   0.0   -0.02   0.06  -0.05  0.02  0.0
  0.02  0.01  0.0   0.0   0.0  0.05  66.63  0.16  6.58  -0.02  -0.01   0.01   0.0   0.02  0.0
 -0.0   0.04  0.0   0.08  0.0  0.01  68.15  0.12  7.2    0.0   -0.04   0.06  -0.08  0.06  0.0
  0.06  0.02  0.02  0.0   0.0  0.08  61.16  0.08  6.52  -0.06  -0.02  -0.02  -0.0   0.1   0.0

Min Loss train: 6.35e-01 val: 5.98e-01
 update plot 7



Loss train: 6.35e-01 val: 5.98e-01 grad: 1.16e+01 lr: 2.0e-03 0.1%┣┫ 2/2.5k [00:27<18:32:08, 27s/it]


...

Loss train: 7.99e-02 val: 8.69e-02 grad: 1.03e+01 lr: 2.5e-04 63.8%┣┫ 1.6k/2.5k [29:03<16:32, 1s/it]
Loss train: 8.25e-02 val: 1.01e-01 grad: 1.54e+01 lr: 2.5e-04 63.8%┣┫ 1.6k/2.5k [29:05<16:31, 1s/it]
Loss train: 8.92e-02 val: 1.08e-01 grad: 8.50e+00 lr: 2.5e-04 63.8%┣┫ 1.6k/2.5k [29:06<16:29, 1s/it]
Loss train: 8.58e-02 val: 1.05e-01 grad: 8.55e+00 lr: 2.5e-04 63.9%┣┫ 1.6k/2.5k [29:07<16:28, 1s/it]
Loss train: 8.35e-02 val: 1.06e-01 grad: 9.24e+00 lr: 2.5e-04 63.9%┣┫ 1.6k/2.5k [29:08<16:27, 1s/it]
Loss train: 8.22e-02 val: 1.03e-01 grad: 1.01e+01 lr: 2.5e-04 64.0%┣┫ 1.6k/2.5k [29:09<16:26, 1s/it]
Loss train: 7.94e-02 val: 1.06e-01 grad: 9.80e+00 lr: 2.5e-04 64.0%┣┫ 1.6k/2.5k [29:10<16:25, 1s/it]
Loss train: 8.10e-02 val: 1.10e-01 grad: 1.01e+01 lr: 2.5e-04 64.0%┣┫ 1.6k/2.5k [29:11<16:24, 1s/it]
Loss train: 7.82e-02 val: 1.05e-01 grad: 8.31e+00 lr: 2.5e-04 64.1%┣┫ 1.6k/2.5k [29:12<16:23, 1s/it]
Loss train: 7.82e-02 val: 1.03e-01 grad: 9.66e+00 lr: 2.5e-04 64.1%┣┫ 1.6k/2.5k [29:13<16:2


 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×15 Matrix{Float64}:
  0.3   0.0   0.0   0.0   0.0  0.0   170.64  0.06  16.19  -0.3    0.02  -0.0    0.27  0.01  0.0
  0.11  0.0   0.0   0.36  0.0  0.01  146.58  0.2   19.82  -0.11   0.29  -0.0   -0.36  0.18  0.0
  0.74  0.2   0.02  0.0   0.0  0.14  150.82  0.42  25.95  -0.74  -0.2   -0.02   0.67  0.28  0.0
 -0.0   0.45  0.23  0.0   0.0  0.08  127.76  0.19  19.6    0.0   -0.45  -0.23   0.48  0.2   0.0
 -0.0   0.0   0.0   0.36  0.0  0.15  191.72  0.02  16.12   0.0   -0.0    0.28  -0.36  0.08  0.0
  0.04  0.06  0.03  0.0   0.0  0.17  183.56  0.05  14.0   -0.04  -0.06  -0.03   0.14  0.0   0.0
 -0.0   0.0   0.0   0.81  0.0  0.34  104.48  0.28  21.49   0.0    0.19   0.25  -0.81  0.36  0.0
  0.07  0.0   0.0   0.7   0.0  0.0   120.13  0.3   21.18  -0.07   0.25   0.08  -0.7   0.44  0.0

Min Loss train: 7.17e-02 val: 7.81e-02
 update plot 4



Loss train: 8.11e-02 val: 7.81e-02 grad: 8.14e+00 lr: 2.5e-04 76.8%┣┫ 1.9k/2.5k [34:59<10:34, 1s/it]
Loss train: 8.28e-02 val: 1.00e-01 grad: 1.35e+01 lr: 2.5e-04 76.8%┣┫ 1.9k/2.5k [35:00<10:33, 1s/it]
Loss train: 8.59e-02 val: 9.95e-02 grad: 9.89e+00 lr: 2.5e-04 76.9%┣┫ 1.9k/2.5k [35:01<10:32, 1s/it]
Loss train: 8.50e-02 val: 1.06e-01 grad: 8.92e+00 lr: 2.5e-04 76.9%┣┫ 1.9k/2.5k [35:02<10:31, 1s/it]
Loss train: 7.73e-02 val: 1.03e-01 grad: 7.17e+00 lr: 2.5e-04 77.0%┣┫ 1.9k/2.5k [35:03<10:30, 1s/it]
Loss train: 7.61e-02 val: 1.03e-01 grad: 9.50e+00 lr: 2.5e-04 77.0%┣┫ 1.9k/2.5k [35:04<10:29, 1s/it]
Loss train: 7.64e-02 val: 1.02e-01 grad: 9.07e+00 lr: 2.5e-04 77.0%┣┫ 1.9k/2.5k [35:05<10:28, 1s/it]
Loss train: 7.51e-02 val: 1.08e-01 grad: 1.00e+01 lr: 2.5e-04 77.1%┣┫ 1.9k/2.5k [35:06<10:27, 1s/it]
Loss train: 7.49e-02 val: 1.07e-01 grad: 7.26e+00 lr: 2.5e-04 77.1%┣┫ 1.9k/2.5k [35:07<10:25, 1s/it]
Loss train: 7.37e-02 val: 9.96e-02 grad: 9.00e+00 lr: 2.5e-04 77.2%┣┫ 1.9k/2.5k [35:08<10:2


 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×15 Matrix{Float64}:
  0.3   0.0   0.0   0.0   0.0  0.0   174.19  0.06  16.52  -0.3    0.02  -0.0    0.27  0.01  0.0
  0.11  0.0   0.01  0.34  0.0  0.01  148.72  0.2   20.28  -0.11   0.29  -0.01  -0.34  0.18  0.0
  0.76  0.21  0.01  0.0   0.0  0.16  153.22  0.43  26.68  -0.76  -0.21  -0.01   0.69  0.28  0.0
 -0.0   0.45  0.22  0.0   0.0  0.09  128.63  0.18  19.94   0.0   -0.45  -0.22   0.5   0.17  0.0
 -0.0   0.0   0.0   0.36  0.0  0.15  195.71  0.02  16.45   0.0   -0.0    0.28  -0.36  0.08  0.0
  0.04  0.06  0.03  0.0   0.0  0.17  187.36  0.05  14.29  -0.04  -0.06  -0.03   0.14  0.0   0.0
 -0.0   0.0   0.0   0.85  0.0  0.35  104.75  0.29  22.06   0.0    0.21   0.25  -0.85  0.38  0.0
  0.07  0.0   0.0   0.71  0.0  0.0   121.43  0.31  21.71  -0.07   0.26   0.07  -0.71  0.45  0.0

Min Loss train: 6.88e-02 val: 6.36e-02
 update plot 7



Loss train: 8.68e-02 val: 6.36e-02 grad: 9.08e+00 lr: 1.3e-04 84.8%┣┫ 2.1k/2.5k [38:39<06:56, 1s/it]
Loss train: 7.72e-02 val: 9.65e-02 grad: 1.65e+01 lr: 1.3e-04 84.8%┣┫ 2.1k/2.5k [38:40<06:55, 1s/it]
Loss train: 8.26e-02 val: 9.34e-02 grad: 7.12e+00 lr: 1.3e-04 84.9%┣┫ 2.1k/2.5k [38:41<06:54, 1s/it]
Loss train: 7.90e-02 val: 9.38e-02 grad: 8.77e+00 lr: 1.3e-04 84.9%┣┫ 2.1k/2.5k [38:42<06:52, 1s/it]
Loss train: 7.56e-02 val: 9.51e-02 grad: 8.23e+00 lr: 1.3e-04 85.0%┣┫ 2.1k/2.5k [38:43<06:51, 1s/it]
Loss train: 7.40e-02 val: 1.01e-01 grad: 7.23e+00 lr: 1.3e-04 85.0%┣┫ 2.1k/2.5k [38:44<06:50, 1s/it]
Loss train: 7.25e-02 val: 1.02e-01 grad: 7.60e+00 lr: 1.3e-04 85.0%┣┫ 2.1k/2.5k [38:45<06:49, 1s/it]
Loss train: 7.33e-02 val: 9.83e-02 grad: 8.87e+00 lr: 1.3e-04 85.1%┣┫ 2.1k/2.5k [38:46<06:48, 1s/it]
Loss train: 7.19e-02 val: 1.01e-01 grad: 8.88e+00 lr: 1.3e-04 85.1%┣┫ 2.1k/2.5k [38:47<06:47, 1s/it]
Loss train: 7.16e-02 val: 9.82e-02 grad: 8.85e+00 lr: 1.3e-04 85.2%┣┫ 2.1k/2.5k [38:48<06:4

In [323]:
expr_name = "5s8r1c-01"
fig_path = string("./results_catalyst/", expr_name, "/figs")
ckpt_path = string("./results_catalyst/", expr_name, "/checkpoint")
@load string(ckpt_path, "/mymodel.bson") p #opt l_loss_train l_loss_val list_grad iter

#p[9] = -2
p_cutoff = 0.0

display_p(p)

for i_exp = 1:n_exp
    cbi(p, i_exp)
end

loss_epoch = zeros(Float64, n_exp);
for i_exp = 1:n_exp
    loss_epoch[i_exp] = loss_neuralode(p, i_exp)
end
loss_train = mean(loss_epoch[l_train])
loss_val = mean(loss_epoch[l_val])
loss_train, loss_val


 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×15 Matrix{Float64}:
 -0.0   0.54  0.01  0.0   0.0  0.0   80.24  0.14   9.29   0.0   -0.54  -0.01   0.5   0.05  0.0
  0.33  0.19  0.05  0.0   0.0  0.01  72.31  0.25   9.66  -0.33  -0.19  -0.05   0.48  0.09  0.0
  0.39  0.0   0.0   0.0   0.0  0.25  95.21  0.24  11.09  -0.39   0.27   0.01   0.1   0.0   0.0
 -0.0   0.0   0.0   0.52  0.0  0.15  89.59  0.14   9.54   0.0    0.13   0.32  -0.52  0.08  0.0
 -0.0   0.0   0.0   0.65  0.0  0.0   77.4   0.27  11.19   0.0    0.41  -0.0   -0.65  0.24  0.0
  0.12  0.0   0.0   0.36  0.0  0.01  91.15  0.28  11.15  -0.12   0.35  -0.0   -0.36  0.12  0.0
 -0.0   0.0   0.0   0.47  0.0  0.07  63.42  0.13   9.59   0.0    0.07   0.17  -0.47  0.23  0.0
  0.29  0.0   0.01  0.0   0.0  0.08  98.2   0.13   9.39  -0.29   0.07  -0.01   0.23  0.01  0.0



(0.08336832189868705, 0.0822633956374589)

In [30]:
# pyplot()
plot_loss(l_loss_train, l_loss_val; yscale = :log10)

"./results_cat3/7s8r1c-08/figs\\loss_grad.png"

In [218]:
function loss_neuralode_res(p)
    l_loss_exp = zeros(n_exp)
    for i_exp in 1:n_exp
        exp_data = l_exp_data[i_exp]
        pred = Array(pred_n_ode(p, i_exp, exp_data))
        masslist = sum(clamp.(@view(pred[1:end-1, :]), 0, Inf), dims = 1)'
        l_loss_exp[i_exp] = mae(masslist, @view(exp_data[1:length(masslist), 3]))
    end
    return l_loss_exp
end

loss_neuralode_res (generic function with 1 method)

In [203]:
println("results after pruning")
maxiters = 1e5
p_cutoff = 0.0

results after pruning


0.0

In [221]:
loss_epoch = zeros(Float64, n_exp);
for i_exp = 1:n_exp
    loss_epoch[i_exp] = loss_neuralode(p, i_exp)
end
loss_train = mean(loss_epoch[l_train])
loss_val = mean(loss_epoch[l_val])
@printf(
    "Loss train: %.2f val: %.2f p_cutoff: %.2e",
    log10(loss_train),
    log10(loss_val),
    p_cutoff
)

display_p(p)

Loss train: -0.59 val: -0.58 p_cutoff: 1.85e-01
 species (column) reaction (row)
w_in | w_cat_in | Ea | b | lnA | w_out | w_cat_out
8×19 Matrix{Float64}:
 -0.0   -0.0  -0.0   -0.0   -0.0  0.0   -0.0  0.0   289.36  0.12  26.97   0.0   0.0   0.0    0.0    0.0   -0.0   0.0   0.0
  1.75  -0.0  -0.0   -0.0    0.0  0.0   -0.0  0.19  238.25  0.56  44.6   -1.75  0.0   0.0    0.0    0.52   1.23  0.0   0.0
 -0.0   -0.0   0.68  -0.0   -0.0  0.0    0.0  0.35  251.77  0.45  43.44   0.0   0.0  -0.68   0.0    0.0    0.1   0.59  0.0
 -0.0   -0.0  -0.0   -0.0   -0.0  0.0   -0.0  0.19  263.4   0.08  28.33   0.0   0.0   0.0    0.0    0.0   -0.0   0.0   0.0
 -0.0   -0.0  -0.0    0.21   0.0  1.21   0.0  0.01  261.82  0.71  49.65   0.0   0.0   0.0   -0.21   0.28  -1.21  1.13  0.0
 -0.0   -0.0   0.21  -0.0    0.6  0.0   -0.0  0.07  279.12  0.67  49.71   0.0   0.0  -0.21   0.0   -0.6    0.81  0.0   0.0
 -0.0   -0.0   0.0   -0.0    0.0  0.53  -0.0  0.16  127.97  0.0   25.21   0.0   0.0   0.27   0.0    0.26  -0

In [222]:
for i_exp in randperm(n_exp)
    cbi(p, i_exp)
end

In [223]:
l_loss_exp = loss_neuralode_res(p)

18-element Vector{Float64}:
 0.5731249928232994
 0.573818698550572
 0.5599319238596631
 0.6038493091687965
 0.6018047217616849
 0.6049788272576078
 0.5073004092208907
 0.42357193899865203
 0.4033434698914744
 0.4233822601722467
 0.5204082517584994
 0.4756764430372351
 0.5732156460521934
 0.5729433777049404
 0.5785797501762748
 0.5167738478487672
 0.5376574507332098
 0.5233036501654168

In [226]:
l_exp_data = []
l_exp_info = zeros(Float64, length(l_exp), 3)
for (i_exp, value) in enumerate(l_exp)
    filename = string("exp_data_cat3/expdata_no", string(value), ".txt")
    exp_data = Float64.(load_exp(filename))
    push!(l_exp_data, exp_data)
    l_exp_info[i_exp, 1] = exp_data[1, 2] # initial temperature, K
end
l_exp_info[:, 2] = readdlm("exp_data_cat3/beta.txt")[l_exp]
#l_exp_info[:, 3] = log.(readdlm("exp_data_cat3/ocen.txt")[l_exp] .+ llb)
l_exp_info[:, 3] = (readdlm("exp_data_cat3/cata_conc.txt")[l_exp]);

l_loss_exp = loss_neuralode_res(p)

conc_end = zeros(n_exp, ns)

18×8 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [ ]:
# Plot TGA data
list_plt = []
for i_exp = 1:18
    T0, beta, cata_conc = l_exp_info[i_exp, :]
    exp_data = l_exp_data[i_exp]
    sol = pred_n_ode(p, i_exp, exp_data)
    Tlist = similar(sol.t)
    T0, beta, cata_conc = l_exp_info[i_exp, :]
    for (i, t) in enumerate(sol.t)
        Tlist[i] = getsampletemp(t, T0, beta)
    end

    conc_end[i_exp, :] .= sol[:, end]

    plt = plot(
        Tlist,
        exp_data[:, 3],
        seriestype = :scatter,
        label = "Exp-$i_exp",
        framestyle = :box,
    )
    plot!(
        plt,
        Tlist,
        sum(sol[1:end-1, :], dims = 1)',
        lw = 2,
        legend = false,
    )
    xlabel!(plt, "Temperature [K]")

    exp_cond = @sprintf("T0:%.0f K", T0)
    exp_cond *= @sprintf("\nbeta:%.0f K/min", beta)

    ann_loc = [0.2, 0.3]
    if i_exp in [6, 7, 14]
        ann_loc = [0.7, 0.4]
    end
    annotate!(plt, xlims(plt)[1] + (xlims(plt)[end] - xlims(plt)[1]) * ann_loc[1],
                   ylims(plt)[1] + (ylims(plt)[end] - ylims(plt)[1]) * ann_loc[2],
                   text(exp_cond, 11))

    ylabel!(plt, "Mass [-] (No. $i_exp)")
    ylims!(plt, (0.0, 1.0))
    plot!(
        plt,
        xtickfontsize = 11,
        ytickfontsize = 11,
        xguidefontsize = 12,
        yguidefontsize = 12,
    )

    push!(list_plt, plt)
end

In [ ]:
plt_all = plot(list_plt..., layout = (7, 2))
plot!(plt_all, size = (800, 1200))
png(plt_all, string(fig_path, "/TGA_mass_summary"))

In [ ]:
# https://github.com/DENG-MIT/Biomass.jl/blob/main/backup/crnn_cellulose_ocen_test.jl
varnames = ["Cellu", "S2", "S3", "Vola"]
for i_exp in 1:n_exp
    T0, beta, ocen = l_exp_info[i_exp, :]
    exp_data = l_exp_data[i_exp]
    sol = pred_n_ode(p, i_exp, exp_data)
    Tlist = similar(sol.t)
    T0, beta, ocen = l_exp_info[i_exp, :]
    for (i, t) in enumerate(sol.t)
        Tlist[i] = getsampletemp(t, T0, beta)
    end
    value = l_exp[i_exp]
    list_plt = []
    scale_factor = 1 ./ maximum(sol[:, :], dims=2)
    scale_factor .= 1.0
    plt = plot(Tlist, clamp.(sol[1, :], 0, Inf), label = varnames[1])
    for i in 2:ns
        if scale_factor[i] > 1.1
            _label =  @sprintf("%s x %.2e", varnames[i], scale_factor[i])
        else
            _label = varnames[i]
        end
        plot!(plt, Tlist, clamp.(sol[i, :], 0, Inf) * scale_factor[i], label = _label)
    end
    xlabel!(plt, "Temperature [K]");
    ylabel!(plt, "Mass (-)");
    plot!(plt, size=(350, 350), legend=:topleft, framestyle=:box)
    plot!(
        plt,
        xtickfontsize = 11,
        ytickfontsize = 11,
        xguidefontsize = 12,
        yguidefontsize = 12,
    )
    png(plt, string(fig_path, "/pred_S_exp_$value"))
end

include("dataset.jl")
gr()